# Dynamic Programming Patterns

### Overview

Dynamic programming is used to solve problems where the solution to the overall problem relies on the solution to overlapping subproblems.

A classic example are the Fibonacci numbers, defined as:

`fib(n) = fib(n-1) + fib(n-2)`.

Since we need to recursively solve for `1...n`, we can tell caching the subproblems will be advantageous.

For example:

```
fib(4) = fib(3) + fib(2)

fib(3) = fib(2) + fib(1)
fib(2) = fib(1) + fib(0)
```

As we can see, `fib(2)` is repeated, as is `fib(1)`.

If the optimal solution to a problem can be found by finding the optimal solution to its subproblems we call this the __optimal substructure property__.

### Dynamic Programming Approaches

#### 1. Top-down with Memoization

With this approach, we set up a cache to store every result. Before jumping into our calculation, we check if an answer exists in the memoization cache.

```python
def calculate_fibonacci(n: int):
  memoization_cache = [-1 for x in range(n+1)]
  return _calculate_fibonacci(memoization_cache, n)


def _calculate_fibonacci(memoization_cache: list[int], n: int) -> int:
  if n < 2:
    return n

    # if we have already solved this subproblem, simply return the result from the cache
  if memoization_cache[n] >= 0:
    return memoization_cache[n]

  memoization_cache[n] = _calculate_fibonacci(memoization_cache, n-1) +
    _calculate_fibonacci(memoization_cache, n-2)
  
    return memoization_cache[n]
```

#### 2. Bottom-up Tabulation

With this approach, we start from the simplest subproblem and then build our way up to the solution by filling in a table.

```python
def calculate_fibonacci(n):
  dp = [0, 1]
  for i in range(2, n + 1):
    dp.append(dp[i - 1] + dp[i - 2])

  return dp[n]
```

#### Conclusion

Whichever technique we choose, we should use the same general approach:

1. Solve the problem using recursion
2. Apply top-down memoization or bottom-up tabulation

Writing out the recursion tree with method calls can often help you spot the solution.

### 0/1 Knapsack

(See other notes.)

### Unbounded Knapsack

These problems involve something like a knapsack but you can take the item __infinite__ times.

__Examples__

* Given a list of profits, weights and capacity, find the maximum profit IF the number of items you can take are unbounded.
* Given a rod of length ‘n’, we are asked to cut the rod and sell the pieces in a way that will maximize the profit. We are also given the price of every piece of length.
* Given an infinite supply of ‘n’ coin denominations and a total money amount, we are asked to find the total number of distinct ways to make up that amount.
* Given an infinite supply of ‘n’ coin denominations and a total money amount, we are asked to find the minimum number of coins needed to make up that amount.
* We are given a ribbon of length ‘n’ and a set of possible ribbon lengths. We need to cut the ribbon into the maximum number of pieces that comply with the above-mentioned possible lengths. Write a method that will return the count of pieces.

#### General Approach

The general approach is to do a DFS with backtracking. In other words, recurse with the item, but don't increase the index, and then recurse without the item, but do increase the index.

Then you can compare these two things or sum them.

```python
def rod_cutting(lengths: list[int], prices: list[int], remaining_length: int) -> int:
    """
    Given a rod of length ‘n’, we are asked to cut the rod and sell the pieces in a way that
    will maximize the profit. We are also given the price of every piece of length ‘i’ where
    ‘1 <= i <= n’.

    Example:
    Lengths: [1, 2, 3, 4, 5]
    Prices: [2, 6, 7, 10, 13]
    Rod Length: 5

    5xlength 1 = 10 profit
    2xlength 2 + 1xlength 1 = 14 profit
    1xlength 3 + 2xlength 2 = 11 profit
    etc.
    """
    return _rod_cutting(lengths, prices, remaining_length, 0)


def _rod_cutting(lengths: list[int], prices: list[int], remaining_length: int, index: int) -> int:
    """Recursive helper function."""
    if index >= len(lengths) or remaining_length <= 0:
        return 0

    profit_with_this_item = 0

    # Try taking this item
    if lengths[index] <= remaining_length:
        profit_with_this_item = prices[index] + _rod_cutting(lengths, prices, remaining_length-lengths[index], index)

    profit_without_this_item = _rod_cutting(lengths, prices, remaining_length, index+1)

    return max(profit_with_this_item, profit_without_this_item)
```

```python
def minimum_coin_change(denominations: list[int], amount: int) -> int:
    """
    Given an infinite supply of ‘n’ coin denominations and a total money amount,
    we are asked to find the minimum number of coins needed to make up that amount.

    Example:
        Denominations: {1, 2, 3}, Amount: 5 -> 2 (2, 3)
    """
    return _minimum_coin_change(denominations, amount, 0)


def _minimum_coin_change(denominations: list[int], amount: int, index: int) -> int:
    """Recursive helper function."""
    if amount == 0:
        return 0

    if index >= len(denominations):
        return math.inf

    num_with_this_coin = math.inf

    if denominations[index] <= amount:
        temp = _minimum_coin_change(denominations, amount-denominations[index], index)
        if temp != math.inf:  # Don't include if we overshot
            num_with_this_coin = temp + 1

    num_without_this_coin = _minimum_coin_change(denominations, amount, index+1)

    return min(num_with_this_coin, num_without_this_coin)
```

#### Optimizing Topdown

Here, we just create a cache of index & target and then check if we have that already when taking the item

```python
def rod_cutting_topdown(lengths: list[int], prices: list[int], remaining_length: int) -> int:
    """
    Given a rod of length ‘n’, we are asked to cut the rod and sell the pieces in a way that
    will maximize the profit. We are also given the price of every piece of length ‘i’ where
    ‘1 <= i <= n’.

    Example:
    Lengths: [1, 2, 3, 4, 5]
    Prices: [2, 6, 7, 10, 13]
    Rod Length: 5

    5xlength 1 = 10 profit
    2xlength 2 + 1xlength 1 = 14 profit
    1xlength 3 + 2xlength 2 = 11 profit
    etc.
    """
    # Top-down memoization, takes O(n*remaining_length) time and space
    dp = [[-1 for _ in range(remaining_length+1)] for _ in range(len(lengths))]
    return _rod_cutting_topdown(lengths, prices, remaining_length, 0, dp)


def _rod_cutting_topdown(
        lengths: list[int], prices: list[int], remaining_length: int, index: int, dp: list[list[int]]) -> int:
    """Recursive helper function."""
    if index >= len(lengths) or remaining_length <= 0:
        return 0

    profit_with_this_item = 0

    # Try taking this item
    if dp[index][remaining_length] == -1:
        if lengths[index] <= remaining_length:
            profit_with_this_item = prices[index] + _rod_cutting_topdown(lengths, prices, remaining_length-lengths[index], index, dp)

        profit_without_this_item = _rod_cutting_topdown(lengths, prices, remaining_length, index+1, dp)

        dp[index][remaining_length] = max(profit_with_this_item, profit_without_this_item)

    return dp[index][remaining_length]
```

```python
def minimum_coin_change_topdown(denominations: list[int], amount: int) -> int:
    """
    Given an infinite supply of ‘n’ coin denominations and a total money amount,
    we are asked to find the minimum number of coins needed to make up that amount.

    Example:
        Denominations: {1, 2, 3}, Amount: 5 -> 2 (2, 3)
    """
    dp = [[-1 for _ in range(amount+1)] for _ in range(len(denominations))]
    return _minimum_coin_change_topdown(denominations, amount, 0, dp)


def _minimum_coin_change_topdown(denominations: list[int], amount: int, index: int, dp: list[list[int]]) -> int:
    """Recursive helper functions."""
    if amount == 0:
        return 1

    if index >= len(denominations):
        return math.inf

    if dp[index][amount] == -1:
        num_with_this_coin = math.inf

        if denominations[index] <= amount:
            temp = _minimum_coin_change(denominations, amount - denominations[index], index)
            if temp != math.inf:  # Don't include if we overshot
                num_with_this_coin = temp + 1

        num_without_this_coin = _minimum_coin_change(denominations, amount, index + 1)

        dp[index][amount] = min(num_with_this_coin, num_without_this_coin)

    return dp[index][amount]
```

#### Optimizing Bottomup

In this case we create a matrix of item index (rows) x target (columns). We check if we should take the row above, or take the value from this row by subtracting target-current value.

```python
def rod_cutting_bottomup(lengths: list[int], prices: list[int], remaining_length: int) -> int:
    """
    Given a rod of length ‘n’, we are asked to cut the rod and sell the pieces in a way that
    will maximize the profit. We are also given the price of every piece of length ‘i’ where
    ‘1 <= i <= n’.

    Example:
    Lengths: [1, 2, 3, 4, 5]
    Prices: [2, 6, 7, 10, 13]
    Rod Length: 5

    5xlength 1 = 10 profit
    2xlength 2 + 1xlength 1 = 14 profit
    1xlength 3 + 2xlength 2 = 11 profit
    etc.
    """
    # Bottom-up memoization, takes O(n*remaining_length) time and space
    rows = len(lengths)

    dp = [[-1 for _ in range(remaining_length+1)] for _ in range(rows)]

    for row in range(rows):
        dp[row][0] = 0  # For 0 length, we have 0 profit

    for item in range(rows):
        for length in range(1, remaining_length+1):
            price_with_this_item = 0
            price_without_this_item = 0
            if lengths[item] <= length:
                price_with_this_item = prices[item] + dp[item][length - lengths[item]]
            if item > 0:
                price_without_this_item = dp[item-1][length]

            dp[item][length] = max(price_with_this_item, price_without_this_item)

    return dp[rows-1][remaining_length]
```

```python
def minimum_coin_change_bottomup(denominations: list[int], amount: int) -> int:
    """
    Given an infinite supply of ‘n’ coin denominations and a total money amount,
    we are asked to find the minimum number of coins needed to make up that amount.

    Example:
        Denominations: {1, 2, 3}, Amount: 5 -> 2 (2, 3)
    """
    dp = [[math.inf for _ in range(amount+1)] for _ in range(len(denominations))]

    for i in range(len(denominations)):
        dp[i][0] = 0

    for index in range(len(denominations)):
        for amt in range(amount+1):
            amount_with_this_item = math.inf

            if index > 0:
                # Get the amount excluding this item.
                dp[index][amt] = dp[index-1][amt]
            if denominations[index] <= amt:
                if dp[index][amt-denominations[index]] != math.inf:
                    # Get the amount with this coin (+1)
                    dp[index][amt] = min(dp[index-1][amt], dp[index][amt-denominations[index]]+1)

    return -1 if dp[len(denominations)-1][amount] == math.inf else dp[len(denominations)-1][amount]

```